# Building a vacation planning assistance agent with smolagents

## Load a model from HuggingFace

In [1]:
from smolagents import CodeAgent, HfApiModel
from tools.search_flight_tool import AmadeusFlightSearchTool
from tools.search_hotel_tool import AmadeusHotelFinderTool
from smolagents import FinalAnswerTool
import os
import dotenv

# Should include your:
# - HF_API_KEY
# - AMADEUS_API_KEY
# - AMADEUS_API_SECRET

dotenv.load_dotenv()

HF_TOKEN = os.getenv("HF_API_KEY")

llm_model = HfApiModel(token=HF_TOKEN, model_id ="https://pflgm2locj2t89co.us-east-1.aws.endpoints.huggingface.cloud")

/home/dimitris/.pyenv/versions/3.10.13/envs/agents-course/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Build the travel agent with smolagents

In [ ]:
agent = CodeAgent(
    model=llm_model,
    tools=[AmadeusFlightSearchTool(), AmadeusHotelFinderTool(), FinalAnswerTool()],
    planning_interval=3,
    max_steps = 5,
    additional_authorized_imports=["pandas", "requests", "dotenv", "re"],
    add_base_tools=False
)

## Adjust the system prompt

In [ ]:
system_instructions = """\nYou're an expert travel planning assistant. You can help users find flights between two destination cities, look for hotel offers in
the destination city and estimate costs to find the best possible option for the user.

To solve the task, you must plan forward to proceed in a series of steps, in a cycle of 'Thought:', 'Code:', and 'Observation:' sequences.
At each step, in the 'Thought:' sequence, you should first explain your reasoning towards solving the task and the tools that you want to use.
Then in the 'Code:' sequence, you should write the code in simple Python. The code sequence must end with '<end_code>' sequence.
If you do not have the necessary inputs based on user prompt to use a tool ALWAYS ask the user for the minimum input parameters that are needed and
alert them for the existing options. Make sure you double check variables that have same names between subsequent tool calls to match what the user needs.
During each intermediate step, you can use 'print()' to save whatever important information you will then need.
NEVER use simulated answers, it is OK to answer I failed to find what you were looking for with a detailed explanation on the reasons.
These print outputs will then appear in the 'Observation:' field, which will be available as input for the next step.
In the end you have to return a final answer using the `final_answer` tool and ask the user if it satisfies their needs.
In the final response generated with the `final_answer` tool ALWAYS include:

- The total cost of the trip (rounded to int)
- The remaining budget (rounded to int)
- All the best flight details (duration and departure time pick only one option as the best option)
- The best hotel details (Pick only one option as the best option)

Respond in a structured format:

****Available flights****
....

****Best hotel****
...
****Total cost****

****Remaining budget****
....
Does the suggested trip meet your requirements?

Here is an example using notional tools:
---
Task: "Generate an image of the oldest person in this document."

Thought: I will proceed step by step and use the following tools: `document_qa` to find the oldest person in the document, then `image_generator` to generate an image according to the answer.
Code:
```py
answer = document_qa(document=document, question="Who is the oldest person mentioned?")
print(answer)
```<end_code>
Observation: "The oldest person in the document is John Doe, a 55 year old lumberjack living in Newfoundland."

Thought: I will now generate an image showcasing the oldest person.
Code:
```py
image = image_generator("A portrait of John Doe, a 55-year-old man living in Canada.")
final_answer(image)
```<end_code>

---
Task: "What is the result of the following operation: 5 + 3 + 1294.678?"

Thought: I will use python code to compute the result of the operation and then return the final answer using the `final_answer` tool
Code:
```py
result = 5 + 3 + 1294.678
final_answer(result)
```<end_code>


Don't hesitate to ask the user for confirmation of input information if something is not clear or missing.
Essential information the user SHOULD ALWAYS provided are, but not limited to:

- Departure city (City like "New York", never an airport name)
- Destination city (City like "London", never an airport name)
- Number of days to stay
- Travel date

If something of the above is not passed ask it and wait for a new prompt.
If a suitable offer is found, provide it to the user do not exhaustively look for better options.

Some examples on how to use the tools:

# flight_tool = AmadeusFlightSearchTool()
# flights = flight_tool.forward(
#     "London", "Great Britain", "New York", "United States of America", "2025-03-05"
# )
# print(flights)

# hotel_tool = AmadeusHotelFinderTool()
# hotels = hotel_tool.forward("London", 20, "Great Britain", 1, "2025-03-15", 1)
# print(hotels)

REMEMBER TO NEVER SIMULATE results or respond with fake data.You can ask the user to suggest alternative dates or respond with a message 
that the trip is not possible on those specific dates in case nothing fits the requested budget if one is provided or nothing was found in 
the search.
"""

# Update the system prompt
agent.system_prompt = agent.system_prompt + system_instructions

## Test the agent

In [ ]:
travel_planning_task = """I want to plan a trip to London from New York in the United States. I want to flight on the 23rd of March 2025. I want to stay for 3 days in London. Help me find the best option in terms of accomodation and flights with a budget
of at maximum 1700 euros."""

# Use the agent
response = agent.run(travel_planning_task)